In [ ]:
%run 'pymol_and_pdb_functions.py'
import itertools
notebook_prefix = 'ks16'

%run 'ks01_Functions_only.ipynb'

In [ ]:
rosetta_folder = os.path.join(analysis_folder, 'Sasha', 'rosetta_runs', '')
predicted_ddG = pd.read_table(rosetta_folder + 'run-170503-results.csv')
predicted_ddG['mut_number'] = predicted_ddG['mut_list_Scer'].apply(lambda s: s.count(':')+1)
WT_score = np.mean([-562.382, -562.380, -562.375])
predicted_ddG['ddG'] = predicted_ddG.apply(lambda x: np.mean([x['ddg_score1'],x['ddg_score2'],x['ddg_score3']]) - WT_score,axis=1)

potential_dict = {}
for i in range(1,13):
    potential_dict['S%s_scaled' %i] =  pd.read_csv('../Katya/NN/residuals/S%s.csv' %i)
potential_with_segments_column = []
for segment_,data_ in potential_dict.items():
    print segment_
    data_copy = data_.copy()
    data_copy['segment'] = segment_
    potential_with_segments_column.append(data_copy)
potential_concat = pd.concat(potential_with_segments_column,axis=0)
potential_and_ddG = pd.merge(predicted_ddG,potential_concat,on='aa_seq')

In [ ]:
from scipy.stats import gaussian_kde

def density_scatter(x,y,ax,s=5):
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)

    # Sort the points by density, so that the densest points are plotted last
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    ax.scatter(x, y, c=z, s=s, edgecolor='')

plt.close('all')
# axes = plt.subplots(nrows=2,ncols=6)
fig = plt.figure(figsize=(12,8))
for i in range(1,13):

    d = potential_and_ddG[potential_and_ddG['segment_x']=='S%s_scaled'%i]
#     plt.scatter(potential_and_ddG['ddG'],potential_and_ddG['fitnessPotential'])
    ax = fig.add_subplot(3,4,i)
#     np.hist2d(d.ddG.values,d.fitnessPotential.values,bins=50,range=[[-10,45],[-30,16]]);
    density_scatter(d.ddG.values,d.fitnessPotential.values,ax)
    ax.set_xlim([-10,45])
    ax.set_ylim([-30,16])
    if i == 9:
        plt.xlabel("ddG",size=14)
        plt.ylabel('fitnessPotential',size=14)
#     if i in [1,2,3,4,5,6,7,8,]:
#         ax.set_xticks([])
#     if i in [2,3,4,6,7,8,10,11,12]:
#         ax.set_yticks([])
#     plt.text(13,-27,"S%s\nSC = %.2f" % (i, stats.spearmanr(d.ddG.values,d.fitnessPotential.values).correlation),color='black',size=16 )
    ax.set_title("Segment %s, corr = %.2f" % (i, stats.spearmanr(d.ddG.values,d.fitnessPotential.values).correlation),size=8,fontweight='bold')
plt.subplots_adjust(hspace =.3)
plt.subplots_adjust(wspace =.3)